# Feature Extraction with timm (ResNet/ViT)

Use `timm` to load a pre-trained CNN/ViT and extract patch-level features for downstream tasks (MIL, clustering, slide-level aggregation).

In [ ]:
# Optional: install timm if missing (safe in Binder/Colab/local)
import importlib, sys
if importlib.util.find_spec('timm') is None:
    get_ipython().system('pip -q install timm')
import timm, torch
print('timm', timm.__version__)

In [ ]:
# Load a pre-trained backbone without the classifier head (features only)
import torchvision.transforms as T
from PIL import Image
import numpy as np, io, os, requests
model = timm.create_model('resnet50', pretrained=True, num_classes=0)
model.eval()
preprocess = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor(), T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
# Try to load a sample image; fallback to a synthetic patch if offline
url = 'https://picsum.photos/seed/histo/256'
try:
    img = Image.open(io.BytesIO(requests.get(url, timeout=5).content)).convert('RGB')
except Exception:
    img = Image.fromarray((np.random.rand(256,256,3)*255).astype('uint8'), 'RGB')
x = preprocess(img).unsqueeze(0)
with torch.no_grad():
    feats = model(x)
feats = feats.squeeze().numpy()
print('Feature shape:', feats.shape)